# Part 1: Assignment for Web Scrapping using Beautiful Soup

In [1]:
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup object created')

soup object created


In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
#table
table_rows = table.find_all('tr')
#table_rows

In [6]:
import pandas as pd
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df.head()
df.shape

(180, 3)

# Delete Boroughs which are not assigned.

In [7]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.shape
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df1 = df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [9]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2
#df2.shape

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


In [10]:
df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [11]:
df3 = df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Remove duplicate borough

In [12]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [13]:
df3.shape

(103, 3)

# Part 2: latitude and the longitude coordinates of each neighborhood using python Geocoder package

In [50]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [14]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="Pune"
country ="India"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

C:\Users\dilip.sah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


latitude is :- 18.521428 
longtitude is:- 73.8544541


# Reading csv file as the geocoder api is not working properly

In [54]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [25]:
dff = pd.read_csv('Geospatial_Coordinates.csv')
#Merge this with earlier dataframe to get Latitude, Longitude data.
result = pd.merge(df3, dff, left_on='PostalCode',right_on='Postal Code')
result = result.drop(['Postal Code'],axis=1)
result.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Step 3: Explore and cluster the neighborhoods in Toronto

In [26]:
!conda install -c conda-forge folium=0.5.0 

Solving environment: ...working... done

# All requested packages already installed.



In [27]:
import pandas as pd
import folium

print('imported pandas & folium')

imported pandas & folium


In [35]:
sorted_df = result.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])
sorted_df.reset_index(inplace=True)
sorted_df = sorted_df.drop(['index'],axis=1)
sorted_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,NorthYork,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
...,...,...,...,...,...
98,M2M,NorthYork,"Willowdale, Newtonbrook",43.789053,-79.408493
99,M1G,Scarborough,Woburn,43.770992,-79.216917
100,M4C,EastYork,Woodbine Heights,43.695344,-79.318389
101,M2P,NorthYork,York Mills West,43.752758,-79.400049


In [37]:
sorted_df.to_csv('sorted_geoloc.csv')

# Build a test set

In [38]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [40]:
# library to handle JSON files

import pandas as pd

import json

sorted_df.to_json(path_or_buf='geo_toronto.json', orient='table')

In [41]:
import pandas as pd
import folium

#grab a random sample from df
subset_of_df = sorted_df.sample(n=11)
map_test = folium.Map(location=[subset_of_df['Latitude'].mean(), 
                                subset_of_df['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples(): #if you cannot 
    map_test.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Borough))

    
#map_test

#open map_test.html in browser
map_test.save("map_test.html")

# if you cannot generate the maps open PGA_map_*.html from the zip file

In [43]:
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)

In [44]:
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]

{'index': 0,
 'PostalCode': 'M1S',
 'Borough': 'Scarborough',
 'Neighbourhood': 'Agincourt',
 'Latitude': 43.7942003,
 'Longitude': -79.2620294}

In [45]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sorted_df['Borough'].unique()),
        sorted_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [47]:
sorted_df.head()
sorted_dataframe = sorted_df

In [49]:
import pandas as pd
import folium

#grab a random sample from df
subset_of_df = sorted_dataframe.sample(n=11)
map_test = folium.Map(location=[subset_of_df['Latitude'].mean(), 
                                subset_of_df['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():#if you cannot 
    map_test.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Borough))

    
#map_test

#open map_test.html in browser
map_test.save("map_test.html")

# if you cannot generate the maps open PGA_map_*.html from the zip file

In [50]:
from folium.plugins import MarkerCluster
map_borough = folium.Map(location=[subset_of_df['Latitude'].mean(), 
 subset_of_df['Longitude'].mean()], 
 zoom_start=10)
mc = MarkerCluster()
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,  row.Longitude],
                 popup=row.Borough))
    map_borough.add_child(mc)


#map_borough

#open in map_borough.html browser 
map_borough.save("map_borough.html")

#if you cannot generate the maps open PGA_map_*.html from the zip file

In [51]:
import pandas as pd
import folium



#grab a random sample from df
toronto_n = sorted_dataframe.sample(n=20)
map_toronto = folium.Map(location=[toronto_n['Latitude'].mean(), 
                                toronto_n['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in toronto_n.itertuples():
    map_toronto.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Neighbourhood))

    
map_toronto 

#open map_toronto.html in browser

map_toronto.save("map_toronto20.html")

#if you cannot generate the maps open PGA_map_*.html from the zip file

In [55]:
address = 'Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\dilip.sah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [52]:
sorted_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,NorthYork,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,"Bedford Park, Lawrence Manor East",43.733283,-79.419750


In [56]:
# create map of Toronto using latitude and longitude values
map_toronto_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(sorted_dataframe['Latitude'], sorted_dataframe['Longitude'], sorted_dataframe['Borough'], sorted_dataframe['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
map_toronto_neighbourhoods

map_toronto_neighbourhoods.save("map_toronto_neighbourhoods.html")

#open map_toronto_neighbourhoods.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

In [57]:
address = 'York, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of York, Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\dilip.sah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinates of York, Toronto are 43.67910515, -79.49118414007154.


# We are doing this for York, Toronto only.

In [58]:
york_data = sorted_dataframe[sorted_dataframe['Borough'] == 'York'].reset_index(drop=True)
york_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
1,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [59]:
# create map of Manhattan using latitude and longitude values
map_york_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york_toronto)  
    
map_york_toronto

map_york_toronto.save("map_york_toronto.html")

#open map_york_toronto.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

In [60]:
neighbourhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = york_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Caledonia-Fairbanks are 43.6890256, -79.453512.


In [62]:
CLIENT_ID='WU2UX203XESJA0HFY55SV5RWZ1BBSKLI32QJYVRJSYA5LFBP'
CLIENT_SECRET='CRCZYN4G5KWRGZPK3BV4ELWGXR1BOY05S1S3U2RGZCMTTCOA'
VERSION='20180323'


In [63]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(

CLIENT_ID, 

CLIENT_SECRET, 

VERSION, 

neighbourhood_latitude, 

neighbourhood_longitude, 

radius, 

LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=WU2UX203XESJA0HFY55SV5RWZ1BBSKLI32QJYVRJSYA5LFBP&client_secret=CRCZYN4G5KWRGZPK3BV4ELWGXR1BOY05S1S3U2RGZCMTTCOA&v=20180323&ll=43.6890256,-79.453512&radius=500&limit=100'

In [64]:
york_results = requests.get(url).json()
york_results

{'meta': {'code': 200, 'requestId': '5ebdaf3c1835dd0028b33654'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.6935256045, 'lng': -79.44730040297749},
   'sw': {'lat': 43.6845255955, 'lng': -79.45972359702252}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4eb0ca53be7bb9c3eeace0d6',
       'name': 'Nairn Park',
       'contact': {},
       'location': {'address': 'Nairn Ave',
        'crossStreet': 'Chudleigh Rd',
        'lat': 43.69065390931841,
        'lng': -79.4562995791835,
        'labeledLatL

In [65]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
york_venues = york_results['response']['groups'][0]['items']
    
york_nearby_venues = json_normalize(york_venues) # flatten JSON

# filter columns
york_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
york_nearby_venues = york_nearby_venues.loc[:, york_filtered_columns]

# filter the category for each row
york_nearby_venues['venue.categories'] = york_nearby_venues.apply(get_category_type, axis=1)

# clean columns
york_nearby_venues.columns = [col.split(".")[-1] for col in york_nearby_venues.columns]

york_nearby_venues.head()

,name,categories,lat,lng
0,Nairn Park,Park,43.690654,-79.456300
1,Fairbanks Pool,Pool,43.691959,-79.448922
2,Fairbank Memorial Park,Park,43.692028,-79.448924


# Explore neighbourhood in Yark.

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [100]:
york_venues = getNearbyVenues(names=york_data['Neighbourhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Caledonia-Fairbanks
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humewood-Cedarvale
Runnymede, The Junction North
Weston


In [69]:
york_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Caledonia-Fairbanks,43.689026,-79.453512,Nairn Park,43.690654,-79.456300,Park
1,Caledonia-Fairbanks,43.689026,-79.453512,Fairbanks Pool,43.691959,-79.448922,Pool
2,Caledonia-Fairbanks,43.689026,-79.453512,Fairbank Memorial Park,43.692028,-79.448924,Park
3,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,Subway,43.690218,-79.474050,Sandwich Place
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,Dollar Tree,43.690296,-79.474667,Discount Store


In [101]:
york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Caledonia-Fairbanks,3,3,3,3,3,3
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",4,4,4,4,4,4
Humewood-Cedarvale,4,4,4,4,4,4
"Runnymede, The Junction North",4,4,4,4,4,4


In [71]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 13 uniques categories.


# Now group neighbourhood to group into clusters.

In [72]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
york_fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[york_fixed_columns]

york_onehot.head()

,Neighbourhood,Bar,Breakfast Spot,Brewery,Bus Line,Convenience Store,Discount Store,Dog Run,Field,Hockey Arena,Park,Pool,Sandwich Place,Trail
0,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,1,0,0,0
3,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0,0,0,0,0,0,0,0,0,0,0,1,0
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0,0,0,0,0,1,0,0,0,0,0,0,0


In [73]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped.head()

,Neighbourhood,Bar,Breakfast Spot,Brewery,Bus Line,Convenience Store,Discount Store,Dog Run,Field,Hockey Arena,Park,Pool,Sandwich Place,Trail
0,Caledonia-Fairbanks,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.666667,0.333333,0.00,0.00
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.25,0.00,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.000000,0.000000,0.25,0.00
2,Humewood-Cedarvale,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.25,0.25,0.000000,0.000000,0.00,0.25
3,"Runnymede, The Junction North",0.00,0.25,0.25,0.25,0.25,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00


In [74]:
num_top_venues = 3

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    york_temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    york_temp.columns = ['venue','freq']
    york_temp = york_temp.iloc[1:]
    york_temp['freq'] = york_temp['freq'].astype(float)
    york_temp = york_temp.round({'freq': 2})
    print(york_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
  venue  freq
0  Park  0.67
1  Pool  0.33
2   Bar  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
               venue  freq
0                Bar  0.25
1  Convenience Store  0.25
2     Discount Store  0.25


----Humewood-Cedarvale----
          venue  freq
0       Dog Run  0.25
1         Field  0.25
2  Hockey Arena  0.25


----Runnymede, The Junction North----
            venue  freq
0  Breakfast Spot  0.25
1         Brewery  0.25
2        Bus Line  0.25




In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 17

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
york_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)

york_neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue


In [78]:
york_neighbourhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

york_neighbourhoods_venues_sorted.head(2)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Caledonia-Fairbanks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
for ind in np.arange(york_grouped.shape[0]):
    york_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

york_neighbourhoods_venues_sorted.head(2)

ValueError: could not broadcast input array from shape (13) into shape (17)

In [80]:
for ind in np.arange(york_grouped.shape[0]):
    york_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

york_neighbourhoods_venues_sorted.head(2)

ValueError: could not broadcast input array from shape (13) into shape (17)

# Used the Folium library to generated maps to visualize neighborhoods on and how they cluster together

In [81]:
# set number of clusters
kclusters = 2

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
york_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
york_kmeans.labels_[0:5] 

array([0, 1, 1, 1])

In [82]:
# add clustering labels
york_neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', york_kmeans.labels_)

york_merged = york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(york_neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M9N,York,Weston,43.706876,-79.518188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# create map
import math
york_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print (cluster)
    if (not math.isnan(cluster)):
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(york_map_clusters)
       
york_map_clusters

york_map_clusters.save("york_map_clusters.html")

#open york_map_clusters.html in browser
#if you cannot generate the maps open PGA_map_*.html from the zip file

In [97]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,York,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
1,York,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,York,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,York,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
